In [1]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
np.set_printoptions(threshold=np.nan, linewidth=140)

import plotly
import plotly.plotly as py
import plotly.graph_objs as go

import torch as t
from torch.autograd import Variable
from torch import Tensor

plotly.offline.init_notebook_mode(connected=True)

In [2]:
def one_hot_encoding(Yraw, K):
    H = np.arange(0, K).reshape(-1,1)
    Y = (H == Yraw).astype(np.float32)
    return Y


# toy dataset
K = 4
X = np.array([[1,1,0,0, 4,3,2,1, 7,6,5,4, 9,7,4,2],
              [8,6,4,3, 8,6,4,2, 6,5,4,3, 4,3,1,3]], dtype=np.float32)
Y = np.array([[0,0,0,0, 1,1,1,1, 2,2,2,2, 3,3,3,3]], dtype=np.float32)

Y = one_hot_encoding(Y, K)

X = Variable(t.from_numpy(X), requires_grad=False)
Y = Variable(t.from_numpy(Y), requires_grad=False)

In [3]:
# initialization
def initialize():
    m = X.data.size()[1]
    l = [ X.data.size()[0], Y.data.size()[0] ]   
    
    b = Variable(t.from_numpy(np.zeros((l[1],1), dtype=np.float32)), requires_grad=True)
    W = Variable(t.from_numpy((np.random.randn(l[1],l[0])*0.01).astype(np.float32)), requires_grad=True)

    return W,b

In [4]:
def forward(X,Y,W,b, cross_entropy=True):
    Z = W @ X + b
    D, _ = t.max(Z, 0, keepdim=True)
    D *= -1
    expZ = t.exp(Z+D)
    sumsexpZ = t.sum(expZ,0, keepdim=True)
    A = expZ / sumsexpZ
    
    if cross_entropy:
        cost = t.mean(-t.sum(Y*t.log(A+1e-8), 0, keepdim=True))
    else:
        cost = None
    return cost, A
    
    
def train(X,Y,W,b, learning_rate=0.01, iterations=1000, report=50):
    for i in range(iterations):
        #forward
        cost, _ = forward(X,Y,W,b)
        #backward
        cost.backward()
        
        dW = W.grad.data
        db = b.grad.data

        W.data -= learning_rate*dW
        b.data -= learning_rate*db
        
        if not i%report or i == iterations-1:
            print(f'[{i:>06}] Training Cost: {cost.data[0]:.7f}')

In [9]:
W,b = initialize()
train(X,Y,W,b, learning_rate=0.01, iterations=200, report=20)

[000000] Training Cost: 1.3724424
[000020] Training Cost: 0.6485040
[000040] Training Cost: 0.3917653
[000060] Training Cost: 0.3459092
[000080] Training Cost: 0.3273662
[000100] Training Cost: 0.2861877
[000120] Training Cost: 0.2287090
[000140] Training Cost: 0.2070865
[000160] Training Cost: 0.2477724
[000180] Training Cost: 0.2301124
[000199] Training Cost: 0.1832312


In [10]:
def predict(X, W, b):
    _, A = forward(X,None,W,b, cross_entropy=False)
    _, H = t.max(A,0,keepdim=True)
    return H
    
    
colors = ['rgba(252, 0, 0, .8)', 'rgba(0, 200, 0, .8)', 'rgba(0, 0, 200, .8)', 'rgba(200, 0, 200, .8)']
dots = []
for k in range(K):
    x = X.data.numpy()[0,k*4:(k+1)*4]
    y = X.data.numpy()[1,k*4:(k+1)*4]
    trace = go.Scatter(x = x, 
                       y = y,
                       name = f'class: {k}',
                       mode = 'markers',
                       marker = { 'size'  : 10,
                                  'color' : colors[k]} )

    dots.append(trace)
    
    

x1, x2 = np.meshgrid(np.arange(0, 10, 0.1),
                     np.arange(0, 10, 0.1))

dp = np.row_stack((x1.ravel(),x2.ravel())).astype(np.float32)
#to tensor
tdp = t.from_numpy(dp)
#to variable
vdp = Variable(tdp, requires_grad=False)


prediction = predict(vdp, W, b)

p = prediction.data.numpy()
p = p.reshape(100,100)


cntr =  {'z': p,
        'colorscale':'Jet',
        'type': u'contour',
        'dx': 0.1,
        'x0': 0,
        'dy': 0.1,
        'y0': 0 }

data = go.Data([go.Contour(cntr)])

data = data + dots

layout = dict(title = 'Decision surfaces visualisation')

fig = dict(data=data, layout=layout)
    
plotly.offline.iplot(fig)